In [44]:
# import 
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as f 
from torch.utils.data import DataLoader 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 

# Set device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load data 
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Hyperparameters 
input_size = 784 
num_classes = 10 
learning_rate = 0.001 
batch_size = 64 
num_epochs = 1 

# Create Fully Connected Network 
class NN(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        
    def forward(self, x): 
        x = f.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10): 
        super(CNN, self).__init__() 
        # same convolution
        # n_out = [(n_in + 2p - k)/s] + 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)
        
    def forward(self, x): 
        x = f.relu(self.conv1(x))
        x = self.pool(x)
        x = f.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        
        return x 

        
# model = NN(784, 10)
# x = torch.rand(64, 784)
# print(model(x).shape)

# Model = CNN() 
# x = torch.rand(64,1,28,28)
# print(Model(x).shape)
    

# Initialize Network
model = NN(input_size=input_size, num_classes=num_classes).to(device)

# Lass and Optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the Network 
for epoch in range(num_epochs): 
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # Get to coorect shape
        # print(data.shape)
        data = data.reshape(data.shape[0], -1)
        
        # forward 
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient desent or adam step
        optimizer.step()

        
        
# Check accuracy 
def check_accuracy(loader, model): 
    if loader.dataset.train: 
        print("Checking accuracy on training data...")
    else:
        print("Checking accuracy on test data...")
    num_correct = 0 
    num_samples = 0 
    model.eval() 
    
    with torch.no_grad(): 
        for x, y in loader: 
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)
            
            scores = model(x)  # NN()
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum() 
            num_samples += predictions.size(0)
            
        print(f'got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100}')
        
    model.train()


In [45]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data...
got 55848 / 60000 with accuracy 93.08
Checking accuracy on test data...
got 9318 / 10000 with accuracy 93.17999999999999
